<a href="https://colab.research.google.com/github/gtrafa/DL_projects_rainsensor/blob/main/rainDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [140]:
from pathlib import Path
import numpy as np
import joblib
from keras.preprocessing import image
from keras.applications import vgg16

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

# adjust image size
from PIL import Image
from keras.models import model_from_json
# data source 
#https://www.kaggle.com/somesh24/multiclass-images-for-weather-classification

In [ ]:
# Path to folders with training data
rain_path = Path("/rain")
not_rain_path = Path("/not_rain")

image_count = len(list(not_rain_path.glob('*.jpg')))
print(image_count)

909


In [ ]:
def adjustImage(img_begins):
  
  ORIG_WIDTH, ORIG_HEIGHT = img_begins.size
  
  diff = (ORIG_HEIGHT - ORIG_WIDTH) // 2
  WIDTH = 64
  HEIGHT = 64

  crop_rect = (0, diff, ORIG_WIDTH, ORIG_HEIGHT - diff)
  final_img = img_begins.crop(crop_rect)
  final_img.thumbnail((WIDTH, HEIGHT), Image.ANTIALIAS)
  return final_img


In [ ]:
images = []
labels = []

# Load all the not_rain images
for img in not_rain_path.glob("*.jpg"):

    # Load the image from disk
    img = image.load_img(img)

    # Convert the image to a numpy array
    #image_array = image.img_to_array(adjustImage(img))

    # Add the image to the list of images
    images.append(adjustImage(img))

    # For each 'not dog' image, the expected value should be 0
    labels.append(0)

In [126]:
images = []
labels = []

# Load all the not_rain images
for img in not_rain_path.glob("*.jpg"):

    # Load the image from disk
    img = image.load_img(img,  target_size=(64, 64))

    # Convert the image to a numpy array
    #image_array = image.img_to_array(adjustImage(img))
    image_array = image.img_to_array(img)


    # Add the image to the list of images
    images.append(image_array)
    #images.append(adjustImage(img))

    # For each 'not dog' image, the expected value should be 0
    labels.append(0)
  

# Load all the dog images
for img in rain_path.glob("*.jpg"):
    # Load the image from disk
    img = image.load_img(img,  target_size=(64, 64))

    # Convert the image to a numpy array
    #image_array = image.img_to_array(adjustImage(img))
    image_array = image.img_to_array(img)

    # Add the image to the list of images
    images.append(image_array)
    #images.append(adjustImage(img))

    # For each 'dog' image, the expected value should be 1
    labels.append(1)



In [127]:
print(len(labels))
print(len(images[0]))


1122
64


In [129]:
# Create a single numpy array with all the images we loaded
x_train = np.array(images)

# Also convert the labels to a numpy array
y_train = np.array(labels)


In [130]:

# Normalize image data to 0-to-1 range
x_train = vgg16.preprocess_input(x_train)

# Load a pre-trained neural network to use as a feature extractor
pretrained_nn = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Extract features for each image (all in one pass)
features_x = pretrained_nn.predict(x_train)

# Save the array of extracted features to a file
joblib.dump(features_x, "x_train.dat")

# Save the matching array of expected values to a file
joblib.dump(y_train, "y_train.dat")

['y_train.dat']

In [132]:
# Load data set
x_train = joblib.load("x_train.dat")
y_train = joblib.load("y_train.dat")


In [133]:
# Create a model and add layers
model = Sequential()

model.add(Flatten(input_shape=x_train.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


In [134]:
# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=['accuracy']
)

# Train the model
model.fit(
    x_train,
    y_train,
    epochs=10,
    shuffle=True
)


Epoch 1/10
36/36 [==============================] - 1s 6ms/step - loss: 4.1461 - accuracy: 0.8064
Epoch 2/10
36/36 [==============================] - 0s 7ms/step - loss: 0.2764 - accuracy: 0.9824
Epoch 3/10
36/36 [==============================] - 0s 6ms/step - loss: 0.0444 - accuracy: 0.9950
Epoch 4/10
36/36 [==============================] - 0s 7ms/step - loss: 0.0154 - accuracy: 0.9973
Epoch 5/10
36/36 [==============================] - 0s 7ms/step - loss: 0.0093 - accuracy: 0.9967
Epoch 6/10
36/36 [==============================] - 0s 7ms/step - loss: 0.0111 - accuracy: 0.9976
Epoch 7/10
36/36 [==============================] - 0s 6ms/step - loss: 0.0048 - accuracy: 0.9964
Epoch 8/10
36/36 [==============================] - 0s 7ms/step - loss: 0.0021 - accuracy: 0.9998
Epoch 9/10
36/36 [==============================] - 0s 7ms/step - loss: 0.0246 - accuracy: 0.9972
Epoch 10/10
36/36 [==============================] - 0s 7ms/step - loss: 0.0174 - accuracy: 0.9976


In [137]:
# Save neural network structure
model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)

# Save neural network's trained weights
model.save_weights("model_weights.h5")


In [141]:
# Load the json file that contains the model's structure
f = Path("model_structure.json")
model_structure = f.read_text()

# Recreate the Keras model object from the json data
model = model_from_json(model_structure)

# Re-load the model's trained weights
model.load_weights("model_weights.h5")

In [ ]:
# versãoo inicial uma imagem por vez

# Load an image file to test, resizing it to 64x64 pixels (as required by this model)
img = image.load_img("/content/test_files/Rain01.jpg", target_size=(64, 64))

# Convert the image to a numpy array
image_array = image.img_to_array(img)

# Add a forth dimension to the image (since Keras expects a bunch of images, not a single image)
images = np.expand_dims(image_array, axis=0)

# Normalize the data
images = vgg16.preprocess_input(images)

# Use the pre-trained neural network to extract features from our test image (the same way we did to train the model)
feature_extraction_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
features = feature_extraction_model.predict(images)

# Given the extracted features, make a final prediction using our own model
results = model.predict(features)

# Since we are only testing one image with possible class, we only need to check the first result's first element
single_result = results[0][0]

# Print the result
print("Likelihood that this image contains rain: {}%".format(int(single_result * 100)))

In [ ]:
# loads one image for initial tests

#Load an image file to test, resizing it to 64x64 pixels (as required by this model)
img = image.load_img("/content/test_files/Rain01.jpg", target_size=(64, 64))
print(img)

<PIL.Image.Image image mode=RGB size=64x64 at 0x7FA9620105D0>


In [150]:
# evaluating testing data
imagess =[]

# Path to folders with testing data
testing_path = Path("/content/test_files_rain")

# Load all the not_rain images
for index, img in enumerate(testing_path.glob("*.jpg"), start=1):

  # Load an image file to test, resizing it to 64x64 pixels (as required by this model)
  img = image.load_img(img, target_size=(64, 64))

  # Convert the image to a numpy array
  image_array = image.img_to_array(img)

  # Add a forth dimension to the image (since Keras expects a bunch of images, not a single image)
  images = np.expand_dims(image_array, axis=0)

  # Add the image to the list of images
  imagess.append(image_array)

# Normalize the data
images = vgg16.preprocess_input(np.array(imagess))

# Use the pre-trained neural network to extract features from our test image (the same way we did to train the model)
feature_extraction_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
features = feature_extraction_model.predict(images)

# Given the extracted features, make a final prediction using our own model
results = model.predict(features)
for ind in range(len(imagess)):
  # Since we are only testing one image with possible class, we only need to check the first result's first element
  single_result = results[ind][0]
  
  # Print the result
  print("Image {}: Likelihood that this image contains rain: {}%".format(ind, int(single_result * 100)))

Image 0: Likelihood that this image contains rain: 100%
Image 1: Likelihood that this image contains rain: 100%
Image 2: Likelihood that this image contains rain: 100%
Image 3: Likelihood that this image contains rain: 100%
Image 4: Likelihood that this image contains rain: 100%
Image 5: Likelihood that this image contains rain: 100%
Image 6: Likelihood that this image contains rain: 100%
